# C3DV0_ex.ipynb
이 파일은 [dolongbien의 깃헙](https://github.com/dolongbien/HumanBehaviorBKU)에서 가져왔음
파일을 돌리려면 아래의 깃헙을 모두 사용해야 함
1. [dolongbien github](https://github.com/dolongbien/HumanBehaviorBKU)
2. [C3D github](https://github.com/facebookarchive/C3D)
- C3D 돌리기
    - build 생성
- `generate_inout_file.py` 사용
    - `fc6`를 뽑는 파일 생성
- `fc6` 추출


In [6]:
# check Utubtu version
!lsb_release -a
# by default, CUDA is enabled under GPU mode of Google Colab
!ls /usr/local

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
bin	   cuda-10.1  _gcs_config_ops.so  LICENSE.txt  setup.cfg  xgboost
cuda	   etc	      include		  man	       share
cuda-10.0  games      lib		  sbin	       src


In [7]:
# optional
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Note


```
As of the CUDA 8 release, gcc 5.3 is fully supported on Ubuntu 16.06 and Fedora 23.
As of the CUDA 9 release, gcc 6 is fully supported on Ubuntu 16.04, Ubuntu 17.04 and Fedora 25.
The CUDA 9.2 release adds support for gcc 7
```



In [8]:
# make g++, nvcc and boost work together (compatible). 
# Better install gcc before anything else
# update-alternatives --install [link] [name] [path] [priority]

!sudo apt update
!sudo apt install g++-5
!sudo update-alternatives --remove-all gcc 
!sudo update-alternatives --remove-all g++

!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

!sudo update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
!sudo update-alternatives --set cc /usr/bin/gcc

!sudo update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
!sudo update-alternatives --set c++ /usr/bin/g++

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 1s (231 kB/s)
Reading package lists... Done
Building dependency tree       
Reading

In [9]:
!sudo apt-get install -f

# install dependencies

!sudo apt-get install -y libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
!sudo apt-get install libatlas-base-dev
!sudo apt-get install -y --no-install-recommends libboost-all-dev
!sudo apt-get install -y libgflags-dev libgoogle-glog-dev liblmdb-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libleveldb-dev is already the newest version (1.20-2).
libprotobuf-dev is already the newest version (3.0.0-9.1ubuntu1).
libsnappy-dev is already the newest version (1.1.7-1).
libhdf5-serial-dev is already the newest version (1.10.0-patch1+docs-4).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-all-dev is already the newest version 

# Note
```
CUDA, OPENCV
• BLAS (Basic Linear Algebra Subprograms): operations like matrix
multiplication, matrix addition, both implementation for CPU(cBLAS) and
GPU(cuBLAS). provided by MKL(INTEL), ATLAS, openBLAS, etc.
• Boost: a c++ library. > Use some of its math functions and shared_pointer.
• glog,gflags provide logging & command line utilities. > Essential for
debugging.
• leveldb, lmdb: database io for your program. > Need to know this for
preparing your own data.
• protobuf: an efficient and flexible way to define data structure. > Need to
know this for defining new layer
```



In [ ]:
# %cd /content
# !git clone https://github.com/dolongbien/C3D.git

In [ ]:
# # gpu 이름 확인 -> Makefile.config, Makefile.config.example에서 sm부분 변경 (notion참고) 
# # 현재 K80(37), T4(75), P4(61), P100(60) 
# from tensorflow.python.client import device_lib 
# device_lib.list_local_devices()

# # CUDA_ARCH := -gencode arch=compute_20,code=sm_20 \
# # 		-gencode arch=compute_20,code=sm_21 \
# # 		-gencode arch=compute_30,code=sm_30 \
# # 		-gencode arch=compute_35,code=sm_35 \
# # 		-gencode=arch=compute_50,code=sm_50 \ 	
# # 		-gencode arch=compute_60,code=sm_60 \
# # 		-gencode arch=compute_75,code=sm_75 \
# # 		-gencode arch=compute_37,code=sm_37 \ 
# # 		-gencode arch=compute_61,code=sm_61 \
# # 		#-gencode=arch=compute_50,code=compute_50

In [10]:
# compile
# 여기에서 build folder 생성
%cd /content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/
!make clean 
!make all -j8

/content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0
/usr/local/cuda/bin/nvcc -ccbin=/usr/bin/g++ -Xcompiler -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode=arch=compute_50,code=sm_50 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61  -c src/caffe/util/math_functions.cu -o build/src/caffe/util/math_functions.cuo
/usr/local/cuda/bin/nvcc -ccbin=/usr/bin/g++ -Xcompiler -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode=arch=compute_50,code

In [ ]:
# !make test

In [ ]:
# # optional
# !make runtest

In [ ]:
# # conv3d_deepnetA_sport1m_iter_1900000를 dropbox에서 다운로드
# %cd /content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction/
# !wget -O conv3d_deepnetA_sport1m_iter_1900000 https://www.dropbox.com/s/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000?dl=0

# Extract feature, check output folder in c3d_sport1m_feature_extraction_video.sh




In [ ]:
# # extract feature
# %cd /content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction
# !sudo bash c3d_sport1m_feature_extraction_video.sh

---



# Abnormal #001\~012(003, 004제외) + #035~044
## - Normal / Abnormal에 따라 코드 고쳐야 함

- f_input, f_output, sh_out의 파일명 수정
- str(0)/str(1)

In [11]:
%cd /content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video/
import os, os.path
import cv2
import glob
import math

f_input = open('input_list_video.txt', 'w')
f_output = open('output_list_video_prefix.txt','w')
sh_out = open('c3d_sport1m_feature_extraction_video.sh', 'w')

interval_v = 16

cmd1 = 'time GLOG_logtosterr=1 ../../build/tools/extract_image_features.bin prototxt/c3d_sport1m_feature_extractor_video.prototxt conv3d_deepnetA_sport1m_iter_1900000'

cmd2 = 'prototxt/output_list_video_prefix.txt fc6-1'

gpu_id = 0
batch_size = 50
batch_num = 1
max_batch_num = 1

# read file path name in alphabet order
file_list_v = sorted(glob.glob("*.mp4"))

# each video, generate input/output file
for index in range(len(file_list_v)):
	
	# file path of video
	loc = file_list_v[index]
	cap=cv2.VideoCapture(loc)
	
	# generate .sh file
	sh_out.write('rm -rf output/Traning_Normal_Part1_FC/' + loc[:-4])
	sh_out.write('\n')
	sh_out.write('mkdir -p output/Traning_Normal_Part1_FC/' + loc[:-4])
	sh_out.write('\n')
	
	# number of frames
	total_frames = int(cap.get(7))
	print(index+1, total_frames, sep='\tframes=')
	
	# calculate batch number
	curr_batch_num = math.ceil(total_frames/batch_size)
	max_batch_num = batch_num
	if curr_batch_num >= max_batch_num:
		batch_num = curr_batch_num
	else:
		batch_num = max_batch_num
	

	counter = 0   

	# generate file
	while (counter <= total_frames-interval_v):
		f_input.write('input/Traning_Normal_Part1_Video/' + loc + ' ' + str(counter) + ' ' + str(1)) # normal : 0 / abnormal : 1
		f_input.write('\n')
		f_output.write('output/Traning_Normal_Part1_FC/' + loc[:-4] + '/' + str(counter).zfill(6) )
		f_output.write('\n')
		counter = counter + interval_v

# write command to bash file
parameter = ' ' + str(gpu_id) + ' ' + str(batch_size) + ' ' + str(batch_num) + ' ' 
sh_out.write(cmd1 + parameter + cmd2)

print('-------- PARAMETER --------')
print('GPU ID is', gpu_id)
print('Batch size is', batch_size)
print('Number of batch is', batch_num)

f_input.close()
f_output.close()
sh_out.close()

/content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
1	frames=689
2	frames=600
3	frames=900
4	frames=1585
5	frames=1799
6	frames=1469
7	frames=1649
8	frames=1229
9	frames=1499
10	frames=1799
11	frames=1799
12	frames=450
13	frames=780
14	frames=1799
15	frames=1199
16	frames=960
17	frames=450
18	frames=299
19	frames=600
20	frames=900
21	frames=300
22	frames=600
23	frames=900
24	frames=600
25	frames=600
26	frames=900
27	frames=1055
28	frames=2070
29	frames=1043
30	frames=1157
31	frames=966
32	frames=1457
33	frames=732
34	frames=771
-------- PARAMETER --------
GPU ID is 0
Batch size is 50
Number of batch is 42


In [12]:
import os
os.getcwd()

'/content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video'

-> 위에서 확인한 경로로 들어가서 생성된 파일 받아가지고 `C3D/C3D-v1.0/examples/c3d_feature_extraction/`기존 파일 변경하기

- c3d_sport1m_feature_extraction_video.sh -> `c3d_feature_extraction`
- input_list_video.txt -> `c3d_feature_extraction/prototxt`
- output_list_video_prefix.txt -> `c3d_feature_extraction/prototxt`

In [14]:
# extract feature
%cd /content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction
!sudo bash c3d_sport1m_feature_extraction_video.sh

/content/gdrive/My Drive/warming-up project/C3D/C3D-v1.0/examples/c3d_feature_extraction
E0918 11:06:01.427371  8004 extract_image_features.cpp:64] Using GPU #0
I0918 11:06:01.745018  8004 net.cpp:76] Creating Layer data
I0918 11:06:01.745048  8004 net.cpp:112] data -> data
I0918 11:06:01.745110  8004 net.cpp:112] data -> label
I0918 11:06:01.745126  8004 video_data_layer.cpp:283] Opening file prototxt/input_list_video.txt
I0918 11:06:01.746971  8004 video_data_layer.cpp:317] A total of 2211 video chunks.
I0918 11:06:03.408708  8004 video_data_layer.cpp:365] output data size: 50,3,16,112,112
I0918 11:06:03.408741  8004 video_data_layer.cpp:387] Loading mean file from sport1m_train16_128_mean.binaryproto
I0918 11:06:04.383412  8004 net.cpp:127] Top shape: 50 3 16 112 112 (30105600)
I0918 11:06:04.383564  8004 net.cpp:127] Top shape: 50 1 1 1 1 (50)
I0918 11:06:04.383579  8004 net.cpp:159] data does not need backward computation.
I0918 11:06:04.383644  8004 net.cpp:76] Creating Layer con